In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

%matplotlib qt

In [2]:
import scipy.fftpack
from scipy.signal import gaussian
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})
%matplotlib qt

In [3]:
matplotlib.rcParams.update({'font.size': 25})

In [4]:
def my_lin(x,a,b):
    return a*x+b

def my_power_law(x,a,b):
    return a*(x**b)

def my_log(x,a,b):
    return a*log(k*(x-x_0))+b

In [5]:
my_color = ['C0','C1','C2','C3','C4']

In [19]:
%run ../../RF_heating/fct_RF_heating

In [7]:
def func(x0, A, B, C, D, t0,E):
    return A*(x0-t0) / (D+C*(x0-t0)**2)**(1/E) + B

In [8]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
ke = 8.987551787e9
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [9]:
N_ions = 1024
n_dt = 1000
# dt = 100e-9/n_dt
dt = 2*pi/(500*2e6*2*pi)

In [10]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=0)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/Simulations/20211129/DC01_RF08/Try00
> Filename : Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_nt1000.dat
> myslashpos | [0, 5, 12, 21, 30, 42, 51, 61]
> slashcond | 6
No points deleted because they were not complete
Total number of data directories 28


In [11]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC01_RF08', 'DC01_RF09', 'DC01_RF10', 'DC01_RF11', 'DC03_RF08', 'DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF08', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF08', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF08', 'DC09_RF09', 'DC09_RF10', 'DC09_RF11', 'DC11_RF08', 'DC11_RF09', 'DC11_RF10', 'DC11_RF11', 'DC18_RF08', 'DC18_RF09', 'DC18_RF10', 'DC18_RF11']
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC03_RF08
005 > DC03_RF09
006 > DC03_RF10
007 > DC03_RF11
008 > DC05_RF08
009 > DC05_RF09
010 > DC05_RF10
011 > DC05_RF11
012 > DC07_RF08
013 > DC07_RF09
014 > DC07_RF10
015 > DC07_RF11
016 > DC09_RF08
017 > DC09_RF09
018 > DC09_RF10
019 > DC09_RF11
020 > DC11_RF08
021 > DC11_RF09
022 > DC11_RF10
023 > DC11_RF11
024 > DC18_RF08
025 > DC18_RF09
026 > DC18_RF10
027 > DC18_RF11
> condition names ['DC', 'RF']
> number of points 28
> N_ions = 1024
> e_GMol = 50


In [12]:
Lan_file = []
VV_file  = []
for i,address in enumerate(all_subdir):
    Lan_file.append([f for f in listdir(address) if isfile(join(address, f)) and "Temp_3D" in f][0])
    VV_file.append([f for f in listdir(address) if isfile(join(address, f)) and "Temp_SimuType0" in f][0])
print(Lan_file,VV_file)

['Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin', 'Temp_3D_Harmo_N1024_T500uK_F0.50D-19Kg_s_5.bin',

In [119]:
# Temperature and dE/dt
# analyse one


point = 13+1 #2+4*4
subdir = all_subdir[point]
print(subdir)

tt, tt1, T_CM, T_aux, dE, S_heating = plot_T_and_dE_S(subdir+'/',Lan_file[point].strip('.bin'),VV_file[point].strip('.dat'),
                                                      flag_plot=1,temp_avg=1,
                                                      fig_name=VV_file[point][22:44],
                                                      xlim1=(-0.2,11.5),ylim1=(5e-4,2e2),ylim2=(-0.9,0.9))
    
    

/home/adrian/RemoteFS/Rivendel/Simulations/20211129/DC07_RF10/Try00
len save_T [112000]
[5.00000000e-07 3.51196237e-04 1.76132793e-03 8.18884690e-05
 2.02156898e+01 3.26295945e+01 1.42387520e+02]


In [64]:
# Temperature and dE/dt
# analyse all

time_save  = []
T_aux_save = []

for point,subdir in enumerate(all_subdir):
    tt, tt1, T_CM, T_aux, dE, S_heating = plot_T_and_dE_S(subdir+'/',Lan_file[point].strip('.bin'),VV_file[point].strip('.dat'),
                                                          flag_plot=0,temp_avg=1,
                                                          fig_name=VV_file[point][22:44],
                                                          xlim1=(-0.2,11.5),ylim1=(5e-4,2e2),ylim2=(-0.9,0.9))
    T_aux_save.append(T_aux[len(tt)-len(tt1):])
time_save = tt1

len save_T [112000]
[5.00000000e-07 2.44923284e-04 1.53623750e-04 5.68068703e-06
 3.44722987e+01 3.30854680e+01 1.58344610e+02]
len save_T [112000]
[5.00000000e-07 2.36419042e-05 1.12609320e-04 2.40605952e-05
 2.51139761e+01 3.15330173e+01 1.57028131e+02]
len save_T [112000]
[5.00000000e-07 2.36363976e-04 3.17751688e-04 6.78209976e-07
 1.74947024e+01 3.29022640e+01 1.62260093e+02]
len save_T [112000]
[5.00000000e-07 2.65996819e-03 5.53939155e-04 1.59939792e-06
 1.17251952e+01 3.00200793e+01 1.54418915e+02]
len save_T [112000]
[5.00000000e-07 6.05475000e-06 2.06027416e-04 8.21352170e-05
 3.45721192e+01 3.12544306e+01 1.49813521e+02]
len save_T [112000]
[5.00000000e-07 4.64500395e-03 3.04461579e-03 1.55495411e-05
 2.51757649e+01 3.15615266e+01 1.52026317e+02]
len save_T [112000]
[5.00000000e-07 2.05576902e-03 3.67635425e-05 1.51044370e-06
 1.89764509e+01 3.26997199e+01 1.53112129e+02]
len save_T [112000]
[5.00000000e-07 7.41274353e-06 1.65901758e-04 1.77726844e-05
 1.15728523e+01 2.99991

In [121]:
T_aux_savebis = reshape(T_aux_save,(4,7,20000),order='F')
all_subdirbis = reshape(all_subdir,(4,7),order='F')
Urf = [53.85,59.23,64.61,70]
Udc = [1,2,3,4,5,7,6]
dt = time_save[5]-time_save[4]
print(dt)

5.000000000000664e-07


In [116]:
figure('test reshape')
semilogy(time_save,T_aux_savebis[2,3])

In [122]:
outfile = 'Temperature_RF_HEATING'
np.savez( outfile, time_step = dt, URF = Urf, UDC = Udc, temperature = T_aux_savebis )